In [ ]:
import os
import random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from pathlib import Path
SEED = 1234
tf.random.set_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
OUTDIR = '/content/outputs'
os.makedirs(OUTDIR, exist_ok=True)
os.makedirs(os.path.join(OUTDIR, 'model'), exist_ok=True)
os.makedirs(os.path.join(OUTDIR, 'tflite'), exist_ok=True)
print('Output dir:', OUTDIR)

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
y_train = y_train.flatten()
y_test = y_test.flatten()
IMAGE_SIZE = (96, 96)
BATCH_SIZE = 64
EPOCHS = 8
NUM_CLASSES = 10
def preprocess_images(x):
    x = tf.image.resize(x, IMAGE_SIZE)
    x = tf.cast(x, tf.float32) / 255.0
    return x
x_train_proc = preprocess_images(x_train)
x_test_proc = preprocess_images(x_test)
fig, axes = plt.subplots(2,5, figsize=(12,5))
for i, ax in enumerate(axes.flat):
    ax.imshow(x_train[i])
    ax.set_title(str(y_train[i]))
    ax.axis('off')
plt.show()

In [ ]:
from tensorflow.keras import layers, models

def build_model(input_shape=(96,96,3), num_classes=10, alpha=0.35):
    base = tf.keras.applications.MobileNetV2(input_shape=input_shape, include_top=False, weights=None, alpha=alpha)
    x = base.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.2)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    model = models.Model(inputs=base.input, outputs=outputs)
    return model

model = build_model(input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3), num_classes=NUM_CLASSES)
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train_proc, y_train))
test_ds = tf.data.Dataset.from_tensor_slices((x_test_proc, y_test))
train_ds = train_ds.shuffle(5000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(os.path.join(OUTDIR, 'model', 'best_model.h5'), save_best_only=True, monitor='val_accuracy'),
    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)
]
history = model.fit(train_ds, epochs=EPOCHS, validation_data=test_ds, callbacks=callbacks)
model.save(os.path.join(OUTDIR, 'model', 'final_model.h5'))
model.save(os.path.join(OUTDIR, 'model', 'saved_model'))
loss, acc = model.evaluate(test_ds)
print(f'Final test loss: {loss:.4f}, test accuracy: {acc:.4f}')

In [ ]:
def representative_data_gen(x_data, image_size, num_samples=200):
    for i in range(min(num_samples, x_data.shape[0])):
        img = x_data[i]
        img = tf.image.resize(img, image_size)
        img = tf.cast(img, tf.float32) / 255.0
        img = tf.expand_dims(img, 0)
        yield [img]
saved_model_dir = os.path.join(OUTDIR, 'model', 'saved_model')
tflite_out = os.path.join(OUTDIR, 'tflite')
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()
open(os.path.join(tflite_out, 'model_float32.tflite'), 'wb').write(tflite_model)
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
open(os.path.join(tflite_out, 'model_dynamic_quant.tflite'), 'wb').write(tflite_model)
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = lambda: representative_data_gen(x_train, IMAGE_SIZE, num_samples=200)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
try:
    tflite_model = converter.convert()
    open(os.path.join(tflite_out, 'model_int8_fullquant.tflite'), 'wb').write(tflite_model)
    print('Saved model_int8_fullquant.tflite')
except Exception as e:
    print('Full integer quantization failed:', e)
for fname in os.listdir(tflite_out):
    fpath = os.path.join(tflite_out, fname)
    print(fname, '-', os.path.getsize(fpath) / 1024.0, 'KB')

In [ ]:
import numpy as np
from pathlib import Path

def evaluate_tflite_model(tflite_path, x_test, y_test, image_size):
    interpreter = tf.lite.Interpreter(model_path=str(tflite_path))
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    dtype = input_details[0]['dtype']
    total = 0
    correct = 0
    for i in range(len(x_test)):
        img = x_test[i]
        if dtype == np.uint8:
            inp = (img * 255).astype(np.uint8)
        else:
            inp = img.astype(np.float32)
        inp = np.expand_dims(inp, 0)
        interpreter.set_tensor(input_details[0]['index'], inp)
        interpreter.invoke()
        out = interpreter.get_tensor(output_details[0]['index'])
        pred = np.argmax(out[0])
        if pred == int(y_test[i]):
            correct += 1
        total += 1
    acc = correct / total
    return acc, correct, total

tflite_dir = Path(tflite_out)
results = {}
import json
for tfile in tflite_dir.glob('*.tflite'):
    acc, c, t = evaluate_tflite_model(tfile, x_test_proc.numpy(), y_test, IMAGE_SIZE)
    results[tfile.name] = {'accuracy': acc, 'correct': int(c), 'total': int(t), 'size_kb': os.path.getsize(tfile) / 1024.0}
    print(f"{tfile.name}: accuracy={acc:.4f} ({c}/{t}) size={results[tfile.name]['size_kb']:.1f} KB")
with open(os.path.join(OUTDIR, 'tflite_metrics.json'), 'w') as f:
    json.dump(results, f, indent=2)
print('Saved metrics to', os.path.join(OUTDIR, 'tflite_metrics.json'))